In [12]:
from typing import Union
from datetime import datetime
import json
from copy import deepcopy

from pathlib import Path
import pandas as pd
import xmltodict

In [2]:
data_path = Path(r"C:\Users\USER\Downloads\Data\Архитектура HypEx.wxml")

In [3]:
type_mapping = {"rectangle": "global", "rounded rectangle": "temporary"}

status_mapping = {"#38761d": "complete", "#0b5394": "in_progress", "#525c61": "planed"}

In [5]:
tasks = extract_tasks(data_path)
tasks

,task,area,type,status,worker
0,Dataset,Dataset,None,None,None
1,ExperimentData,Dataset,global,complete,None
2,Функции управления и атрибуты,Dataset,global,complete,None
3,update,Dataset,temporary,in_progress,🦊
4,Обёртка groupby в Dataset,Dataset,temporary,in_progress,🦊
5,Функции создания,Dataset,temporary,complete,None
6,ID,Dataset,global,complete,None
7,ID-Name,Dataset,global,complete,None
8,Читаемый Hash,Dataset,global,complete,None
9,Building indexing,Dataset,global,complete,None


In [6]:
tasks.to_excel("tasks.xlsx", index=False, sheet_name="tasks")

In [7]:
stat = get_stat(tasks)
stat.to_excel("tasks.xlsx", index=False, sheet_name="stat")
stat

,date,opened tasks,🦊,🤖
0,2024-03-15 08:34:20.991237,23,7,7


In [8]:
class MTConfig:
    def read_json(self, path: Path):
        with open(path, "r", encoding="utf-8") as f:
            config = json.load(f)

        self.__init__(**config)

    def __init__(
        self,
        config_path: Path = None,
        excel_path: Path = None,
        mindmap_path: Path = None,
        color_status_mapping: dict = None,
        shape_type_mapping: dict = None,
    ):
        """
        Initialize the configuration with given parameters,
        or load from a json file at path provided in constructor call (default to current directory).
        """
        self.excel_path = excel_path
        self.mindmap_path = mindmap_path
        self.color_status_mapping = color_status_mapping
        self.shape_type_mapping = shape_type_mapping

        if config_path is not None:
            read_json(config_path)

    def to_json(self, path: Path = None, return_str: bool = True):
        output = deepcopy(self.__dict__)
        output = {k: str(v) for k, v in output.items()}
        if path:
            with open(path, "w", encoding="utf-8") as f:
                json.dump(output, f)
        if return_str:
            return json.dumps(output)

    def __str__(self):
        return str(self.__dict__)

    def __repr__(self):
        return str(self)

In [9]:
config = MTConfig(
    excel_path="tasks.xlsx",
    mindmap_path=data_path,
    color_status_mapping=status_mapping,
    shape_type_mapping=type_mapping,
)
config.to_json("config.json")

'{"excel_path": "tasks.xlsx", "mindmap_path": "C:\\\\Users\\\\USER\\\\Downloads\\\\Data\\\\\\u0410\\u0440\\u0445\\u0438\\u0442\\u0435\\u043a\\u0442\\u0443\\u0440\\u0430 HypEx.wxml", "color_status_mapping": "{\'#38761d\': \'complete\', \'#0b5394\': \'in_progress\', \'#525c61\': \'planed\'}", "shape_type_mapping": "{\'rectangle\': \'global\', \'rounded rectangle\': \'temporary\'}"}'

In [10]:
config.__dict__.items()

dict_items([('excel_path', 'tasks.xlsx'), ('mindmap_path', WindowsPath('C:/Users/USER/Downloads/Data/Архитектура HypEx.wxml')), ('color_status_mapping', {'#38761d': 'complete', '#0b5394': 'in_progress', '#525c61': 'planed'}), ('shape_type_mapping', {'rectangle': 'global', 'rounded rectangle': 'temporary'})])

In [11]:
class MindTaska:
    def __init__(self, config: Union[MTConfig, Path] = None):
        self.config = config if isinstance(config, MTConfig) else MTConfig(config)
        self.tasks = pd.read_excel(self.config.excel_path, sheet_name="tasks")

    def _parse_node(self, tasks: list, node: dict, project: str, parent: dict = None):
        parent = parent or {}
        result = None
        try:
            result = {
                "task": node["@text"],
                "project": project,
                "type": self.config.shape_type_mapping.get(node.get("@shape"))
                or parent.get("type"),
                "status": self.config.color_status_mapping.get(node.get("@bgColor"))
                or parent.get("status"),
                "worker": node.get("eicon", {}).get("@id"),
            }
            tasks.append(result)
        except KeyError as e:
            print(e)
            print(f"In node :\n{node}")

        topic = node.get("topic", [])
        topic = topic if isinstance(topic, list) else [topic]
        for t in topic:
            self._parse_node(tasks, t, project, result)

    def extract_tasks(self) -> pd.DataFrame:
        with open(self.config.maindmap_path, "r", encoding="utf-8") as f:
            xml_string = f.read()
        xml_data = xmltodict.parse(xml_string)
        tasks = []
        main_node = xml_data["map"]["topic"]["topic"]

        for project in main_node:
            self._parse_node(tasks, project, project["@text"])
        return pd.DataFrame(tasks)

    def get_stat(tasks: pd.DataFrame):
        stat = {
            "date": datetime.now(),
            "opened tasks": tasks[tasks["status"] != "complete"].shape[0],
        }

        for w in tasks["worker"].unique():
            if w is not None:
                stat[w] = tasks[
                    (tasks["worker"] == w) & (tasks["status"] != "complete")
                ].shape[0]

        return pd.DataFrame([stat])
    
    def diff_stat()